In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from scipy import stats

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/classification/speech')

In [ ]:
data_meal = pd.read_excel('features.xlsx', sheet_name="Meal", engine='openpyxl')

In [ ]:
subject_id = np.array(data_meal["Subject_ID"])

In [ ]:
X = data_meal.iloc[:,1:12].to_numpy()
y = np.array([0 if x=='no_rad' else 1 for x in data_meal.iloc[:,-1]])

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X,y)

In [ ]:
model = SVC(kernel='linear')

In [ ]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)
splits = k_fold.split(X,y)

In [ ]:
predictions = list()
labels = []
subjects = []
score = []
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)
splits = k_fold.split(X,y)
for train_indexes, test_indexes in splits:
        model.fit(X[train_indexes], y[train_indexes])
        predictions.append(model.predict(X[test_indexes]))
        labels.append(y[test_indexes])
        subjects.append(subject_id[test_indexes])
        score.append(model.decision_function(X[test_indexes]))

In [ ]:
scores = cross_val_score(model, X, y, scoring='accuracy', cv=splits, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
meal_class = []
k=0
while k<5:
    for x, y,z,w in zip(predictions[k], labels[k], subjects[k], score[k]):
        meal_class.append((x,y,z,w))
    k = k+1


In [ ]:
meal_class = pd.DataFrame(meal_class, columns =['prediction', 'label', 's_id', 'decision_score'])


In [ ]:
meal_class = meal_class.set_index(meal_class['s_id'])

In [ ]:
meal_class['decision_score'] = meal_class['decision_score'].abs()

In [ ]:
data_play = pd.read_excel('features.xlsx', sheet_name="Play", engine='openpyxl')

In [ ]:
subject_id = np.array(data_play["Subject_ID"])

In [ ]:
X = data_play.iloc[:,1:12].to_numpy()
y = np.array([0 if x=='no_rad' else 1 for x in data_play.iloc[:,-1]])

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X,y)

In [ ]:
predictions = list()
labels = []
subjects = []
score = []
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)
splits = k_fold.split(X,y)
for train_indexes, test_indexes in splits:
        model.fit(X[train_indexes], y[train_indexes])
        predictions.append(model.predict(X[test_indexes]))
        labels.append(y[test_indexes])
        subjects.append(subject_id[test_indexes])
        score.append(model.decision_function(X[test_indexes]))

In [ ]:
play_class = []
k=0
while k<5:
    for x, y,z,w in zip(predictions[k], labels[k], subjects[k], score[k]):
        play_class.append((x,y,z,w))
    k = k+1


In [ ]:
play_class = pd.DataFrame(play_class, columns =['prediction_play', 'label_play', 's_id', 'decision_score_play'])
play_class = play_class.set_index(play_class['s_id'])

In [ ]:
play_class['decision_score_play'] = play_class['decision_score_play'].abs()

In [ ]:
final = pd.concat([meal_class, play_class], axis=1)

In [ ]:
disagreement = []
for index, row in final.iterrows():
    if row['prediction'] != row['prediction_play']:
        disagreement.append(index)

In [ ]:
disagreement.remove(1195)

In [ ]:
disagreement

In [ ]:
rank_meal = meal_class.sort_values(by='decision_score', ascending=False)


In [ ]:
rank_meal = rank_meal.drop('s_id', axis=1)

In [ ]:
rank_meal = rank_meal.reset_index()

In [ ]:
rank_play = play_class.sort_values(by='decision_score_play', ascending=False)

In [ ]:
rank_play = rank_play.drop('s_id', axis=1)
rank_play = rank_play.reset_index()

In [ ]:
rank_play

In [ ]:
rank_meal

In [ ]:
disagreement

In [ ]:
len(disagreement)